In [ ]:
from white_noise import add_measurement_noise
from red_noise import add_red_noise, add_gwb

from simulate import load_pulsar, load_from_directories
import matplotlib.pyplot as plt
from simulate import make_ideal
import pint
import sys

import numpy as np

from loguru import logger
pint.logging.setup(sink=sys.stderr, level="WARNING", usecolors=True)

%load_ext autoreload
%autoreload 2